In [68]:
%pip install --quiet --upgrade langchain langchain-community langchain-chroma langchain_experimental yfiles_jupyter_graphs neo4j langchain-openai wikipedia

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.6/15.6 MB 80.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 63.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 52.3 MB/s eta 0:00:00


In [79]:
from langchain_core.runnables import (
    RunnableBranch,
    RunnableLambda,
    RunnableParallel,
    RunnablePassthrough,
)
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import Tuple, List, Optional
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser
import os
from langchain_community.graphs import Neo4jGraph
from langchain.document_loaders import WikipediaLoader
from langchain.text_splitter import TokenTextSplitter
from langchain_openai import ChatOpenAI
from langchain_experimental.graph_transformers import LLMGraphTransformer
from neo4j import GraphDatabase
from yfiles_jupyter_graphs import GraphWidget
from langchain_community.vectorstores import Neo4jVector
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars
from langchain_core.runnables import ConfigurableField, RunnableParallel, RunnablePassthrough


/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3553: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [80]:
import os, getpass
os.environ["LANGCHAIN_TRACING_V2"] = "true"
print("Enter LangChain API Key")
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()
os.environ['LANGCHAIN_ENDPOINT']="https://api.smith.langchain.com"
os.environ['LANGCHAIN_PROJECT']="GraphRag"

print("Enter OpenAI API Key")
os.environ['OPENAI_API_KEY']=getpass.getpass()

os.environ["NEO4J_URI"] = ""
os.environ["NEO4J_USERNAME"] = "neo4j"
print("Enter neo4j API Key")
os.environ["NEO4J_PASSWORD"] = getpass.getpass()



Enter LangChain API Key


KeyboardInterrupt: Interrupted by user

In [11]:
graph = Neo4jGraph()

In [27]:
def get_wiki_docs(queries:list)->list:
  docs=[]
  for query in queries:
    loader=WikipediaLoader(query=query)
    docs.append(loader.load())
  return docs

In [19]:
from langchain_text_splitters import TokenTextSplitter

In [44]:
docs=get_wiki_docs(['Elizabeth I','Elizabeth II','Prince Philip, Duke of Edinburgh'])
text_splitter=TokenTextSplitter(chunk_size=512,chunk_overlap=24)

/usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


In [55]:
chunks=[]
for doc in docs:
  chunks+=text_splitter.split_documents(doc)

In [61]:
llm=ChatOpenAI(model='gpt-4o-mini')

In [63]:
graph_transformer=LLMGraphTransformer(llm=llm)

In [64]:
graph_docs=graph_transformer.convert_to_graph_documents(chunks)

In [65]:
graph.add_graph_documents(graph_docs,baseEntityLabel=True,include_source=True)

In [70]:
from google.colab import output
output.enable_custom_widget_manager()

In [73]:
from yfiles_jupyter_graphs import GraphWidget
default_cypher = "MATCH (s)-[r:!MENTIONS]->(t) RETURN s,r,t LIMIT 300"

def showGraph(cypher: str = default_cypher):
    # create a neo4j session to run queries
    driver = GraphDatabase.driver(
        uri = os.environ["NEO4J_URI"],
        auth = (os.environ["NEO4J_USERNAME"],
                os.environ["NEO4J_PASSWORD"]))
    session = driver.session()
    widget = GraphWidget(graph = session.run(cypher).graph())
    widget.node_label_mapping = 'id'
    #display(widget)
    return widget

showGraph()

GraphWidget(layout=Layout(height='800px', width='100%'))

### Click the link below to view the graph

###https://www.yworks.com/yed-live/?file=https://gist.githubusercontent.com/MetalTanuj/9a3b8abf61cf22328f258c9bd327b336/raw/5b7873d0284b74a5b6fea31c31926209a7201f7c/Imported%20From%20Jupyter%20Notebook

Support for third party widgets will remain active for the duration of the session. To disable support:

In [ ]:
from google.colab import output
output.disable_custom_widget_manager()

In [74]:
from langchain_openai import OpenAIEmbeddings

In [75]:
from langchain_community.vectorstores import Neo4jVector

In [77]:
vector_index=Neo4jVector.from_existing_graph(
    OpenAIEmbeddings(),
    search_type='hybrid',
    node_label='Document',
    text_node_properties=['text'],
    embedding_node_property="embeding"
)

In [81]:
# Retriever

graph.query(
    "CREATE FULLTEXT INDEX entity IF NOT EXISTS FOR (e:__Entity__) ON EACH [e.id]")

# Extract entities from text
class Entities(BaseModel):
    """Identifying information about entities."""

    names: List[str] = Field(
        ...,
        description="All the person, organization, or business entities that "
        "appear in the text",
    )

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are extracting organization and person entities from the text.",
        ),
        (
            "human",
            "Use the given format to extract information from the following "
            "input: {question}",
        ),
    ]
)

entity_chain = prompt | llm.with_structured_output(Entities)

In [82]:
entity_chain.invoke({"question": "Where was Elizabeth II born?"}).names

['Elizabeth II']

In [83]:
def generate_full_text_query(input: str) -> str:
    """
    Generate a full-text search query for a given input string.

    This function constructs a query string suitable for a full-text search.
    It processes the input string by splitting it into words and appending a
    similarity threshold (~2 changed characters) to each word, then combines
    them using the AND operator. Useful for mapping entities from user questions
    to database values, and allows for some misspelings.
    """
    full_text_query = ""
    words = [el for el in remove_lucene_chars(input).split() if el]
    for word in words[:-1]:
        full_text_query += f" {word}~2 AND"
    full_text_query += f" {words[-1]}~2"
    return full_text_query.strip()

# Fulltext index query
def structured_retriever(question: str) -> str:
    """
    Collects the neighborhood of entities mentioned
    in the question
    """
    result = ""
    entities = entity_chain.invoke({"question": question})
    for entity in entities.names:
        response = graph.query(
            """CALL db.index.fulltext.queryNodes('entity', $query, {limit:2})
            YIELD node,score
            CALL {
              WITH node
              MATCH (node)-[r:!MENTIONS]->(neighbor)
              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output
              UNION ALL
              WITH node
              MATCH (node)<-[r:!MENTIONS]-(neighbor)
              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output
            }
            RETURN output LIMIT 50
            """,
            {"query": generate_full_text_query(entity)},
        )
        result += "\n".join([el['output'] for el in response])
    return result

In [87]:
def retriever(question: str):
    print(f"Search query: {question}")
    structured_data = structured_retriever(question)
    unstructured_data = [el.page_content for el in vector_index.similarity_search(question)]
    final_data = f"""Structured data:
{structured_data}
Unstructured data:
{"#Document ". join(unstructured_data)}
    """
    return final_data

In [85]:
# Condense a chat history and follow-up question into a standalone question
_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question,
in its original language.
Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""  # noqa: E501
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)

def _format_chat_history(chat_history: List[Tuple[str, str]]) -> List:
    buffer = []
    for human, ai in chat_history:
        buffer.append(HumanMessage(content=human))
        buffer.append(AIMessage(content=ai))
    return buffer

_search_query = RunnableBranch(
    # If input includes chat_history, we condense it with the follow-up question
    (
        RunnableLambda(lambda x: bool(x.get("chat_history"))).with_config(
            run_name="HasChatHistoryCheck"
        ),  # Condense follow-up question and chat into a standalone_question
        RunnablePassthrough.assign(
            chat_history=lambda x: _format_chat_history(x["chat_history"])
        )
        | CONDENSE_QUESTION_PROMPT
        | ChatOpenAI(temperature=0)
        | StrOutputParser(),
    ),
    # Else, we have no chat history, so just pass through the question
    RunnableLambda(lambda x : x["question"]),
)

In [88]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
Use natural language and be concise.
Answer:"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
    RunnableParallel(
        {
            "context": _search_query | retriever,
            "question": RunnablePassthrough(),
        }
    )
    | prompt
    | llm
    | StrOutputParser()
)

In [89]:
chain.invoke({"question": "Which house did Elizabeth II belong to?"})

Search query: Which house did Elizabeth II belong to?


'Elizabeth II belonged to the House of Windsor.'

In [90]:
chain.invoke(
    {
        "question": "When was she born?",
        "chat_history": [("Which house did Elizabeth I belong to?", "House Of Tudor")],
    }
)

Search query: When was Elizabeth I born?


'Elizabeth I was born on 7 September 1533.'

In [91]:
chain.invoke(
    {
        "question": "Who were her children?",
        "chat_history": [("Which house did Elizabeth II belong to?", "House Of Windsor"),
                         ("When was she born?", "Elizabeth I was born on 7 September 1533.")],
    }
)

Search query: Who were Elizabeth II's children?


'Elizabeth II had four children: Charles, Anne, Andrew, and Edward.'

In [92]:
chain.invoke(
    {
        "question": "List her mother's children?",
        "chat_history": [("Which house did Elizabeth II belong to?", "House Of Windsor"),
                         ("When was she born?", "Elizabeth I was born on 7 September 1533."),
                         ('Who were her children?','Elizabeth II had four children: Charles, Anne, Andrew, and Edward.')],
    }
)

Search query: Who were Elizabeth II's mother's children?


"Elizabeth II's mother, Queen Elizabeth The Queen Mother, had two children: Elizabeth II and Princess Margaret."

In [100]:
from langchain_chroma import Chroma
vectorstore=Chroma.from_documents(chunks,embedding=OpenAIEmbeddings())
retriever=vectorstore.as_retriever()
from langchain import hub
prompt=hub.pull("rlm/rag-prompt")

In [111]:
chain.invoke({
    "question":"Who was the influencer of William Shakespeare?"
})

Search query: Who was the influencer of William Shakespeare?


'James VI and I was the influencer of William Shakespeare.'

In [110]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)
rag_chain=(
    {"context": retriever | format_docs,"question":RunnablePassthrough()}
    | prompt
    | llm
    |StrOutputParser()
)
rag_chain.invoke("Who was the influencer of William Shakespeare?")

"The retrieved context does not provide specific information about an influencer of William Shakespeare. Therefore, I don't know the answer."

In [113]:
question='Who claimed the authority of Kingdom of France'

## Comparison of only vector based RAG and Hybrid Graph RAG

### Vector Based

In [114]:
rag_chain.invoke("question")

"I don't know."

### Hybrid Graph RAG

In [115]:
chain.invoke({
    "question":f"{question}"
})

Search query: Who claimed the authority of Kingdom of France


'Henry VIII claimed authority over the Kingdom of France.'